### XMLファイルを出力

In [131]:
# モジュールをインポート
from xml.dom import minidom
import xmljson
import xml.etree.ElementTree as ElementTree
import os

In [138]:
# 時刻の計算
hour = 8
begin_second = float(3600 * hour)
end_second = float(begin_second + 600)

In [139]:
# ファイルパスの設定
# カレントディレクトリは 'jupyter'
# file_name = 'local'
file_name = 'transit'
file_path = '../../DUARoutes/' + file_name + '.rou.xml'
print(os.path.exists(file_path))

#パス、ファイル名
save_path_file = '../../routes/' + str(hour) + 'am/' + file_name + '.rou.xml'

True


In [140]:
# xmlデータを読み込み
read_tree = ElementTree.parse(file_path)
# Root階層を取得
xml_root = read_tree.getroot()
# xmlをdictに変換
xml_dict = xmljson.yahoo.data(xml_root)

In [141]:
# 書き込みファイル用のdomを作成
dom = minidom.Document()

# rootノードの生成
routes = dom.createElement("routes")
routes_attr = dom.createAttribute('xmlns:xsi')
routes_attr.value = 'http://www.w3.org/2001/XMLSchema-instance'
routes.setAttributeNode(routes_attr)
routes_attr = dom.createAttribute('xsi:noNamespaceSchemaLocation')
routes_attr.value = 'http://sumo.dlr.de/xsd/routes_file.xsd'
routes.setAttributeNode(routes_attr)

# domに子ノードとして追加
dom.appendChild(routes)

<DOM Element: routes at 0x1272b0670>

In [142]:
# 車両データの追加
for xml_vehicle in xml_dict['routes']['vehicle']:

    if begin_second <= float(xml_vehicle['depart']) < end_second:
        depart_time = float(xml_vehicle['depart']) - begin_second
        # vehicleのD要素を作成
        vehicle = dom.createElement('vehicle')

        # id属性の追加
        vehicle_attr = dom.createAttribute('id')
        vehicle_attr.value = xml_vehicle['id']
        vehicle.setAttributeNode(vehicle_attr)

        # type属性の追加
        vehicle_attr = dom.createAttribute('type')
        vehicle_attr.value = xml_vehicle['type']
        vehicle.setAttributeNode(vehicle_attr)

        # depart属性の追加
        vehicle_attr = dom.createAttribute('depart')
        vehicle_attr.value = str(depart_time) + '0'
        vehicle.setAttributeNode(vehicle_attr)

        # departSpeed属性の追加
        vehicle_attr = dom.createAttribute('departSpeed')
        vehicle_attr.value = xml_vehicle['departSpeed']
        vehicle.setAttributeNode(vehicle_attr)

        if 'local' == file_name:
            # departPos属性の追加
            vehicle_attr = dom.createAttribute('departPos')
            vehicle_attr.value = xml_vehicle['departPos']
            vehicle.setAttributeNode(vehicle_attr)

            # arrivalSpeed属性の追加
            vehicle_attr = dom.createAttribute('arrivalSpeed')
            vehicle_attr.value = xml_vehicle['arrivalSpeed']
            vehicle.setAttributeNode(vehicle_attr)

            # arrivalPos属性の追加
            vehicle_attr = dom.createAttribute('arrivalPos')
            vehicle_attr.value = xml_vehicle['arrivalPos']
            vehicle.setAttributeNode(vehicle_attr)

        # route要素とedges属性の追加
        route = dom.createElement('route')
        route_attr = dom.createAttribute('edges')
        route_attr.value = xml_vehicle['route']['edges']
        route.setAttributeNode(route_attr)
        vehicle.appendChild(route)

        # routes要素にvehicle要素を追加
        routes.appendChild(vehicle)

In [143]:
#整形、出力
xml_str = dom.toprettyxml(indent="\t")

#ファイルの読み書き
with open(save_path_file, "w") as f:
    f.write(xml_str)